In [1]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from data_loader_RVAE import sentence_to_tensor, tensor_to_sentence

class RVAE(nn.Module):
    N_PITCH = 12
    N_MAIN_QUALITY = 3 # A changer aussi dans data_loader
    N_EXTRA_QUALITY = 3
    N_HIDDEN = 256
    N_LATENT = 16
    N_INPUT = N_PITCH * N_MAIN_QUALITY + N_EXTRA_QUALITY + 3
    rnn_type= 'gru'
    # word_dropout= 0.75
    # pad_idx=
    # unk_idx=

    N_LAYERS=1
    bidirectional=False
    def __init__(self, max_sequence_length):
        
        
        super(RVAE, self).__init__()
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        self.max_sequence_length = max_sequence_length
        # self.sos_idx = sos_idx
        # self.eos_idx = eos_idx
        # self.pad_idx = pad_idx
        # self.unk_idx = unk_idx

        # self.latent_size = latent_size

        # self.rnn_type = rnn_type
        # self.bidirectional = bidirectional
        # self.num_layers = num_layers
        # self.hidden_size = hidden_size

        # self.word_dropout_rate = word_dropout
        # self.embedding_dropout = nn.Dropout(p=embedding_dropout)

        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU

        else:
            raise ValueError()

        self.encoder_rnn = rnn(self.N_INPUT, self.N_HIDDEN, num_layers=self.N_LAYERS, bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(self.N_INPUT, self.N_HIDDEN, num_layers=self.N_LAYERS, bidirectional=self.bidirectional,
                               batch_first=True)

        self.hidden_factor = (2 if self.bidirectional else 1) * self.N_LAYERS

        self.hidden2mean = nn.Linear(self.N_HIDDEN * self.hidden_factor, self.N_LATENT)
        self.hidden2logv = nn.Linear(self.N_HIDDEN * self.hidden_factor, self.N_LATENT)
        self.latent2hidden = nn.Linear(self.N_LATENT, self.N_HIDDEN * self.hidden_factor)
        self.outputs2pred = nn.Linear(self.N_HIDDEN * (2 if self.bidirectional else 1), self.N_INPUT)
        

    def forward(self, input_sequence, length):

        batch_size = input_sequence.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = input_sequence[sorted_idx]

        # ENCODER
        
        packed_input = rnn_utils.pack_padded_sequence(input_sequence, sorted_lengths.data.tolist(), batch_first=True)

        _, hidden = self.encoder_rnn(packed_input)

        if self.bidirectional or self.N_LAYERS > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.N_HIDDEN*self.hidden_factor)
        else:
            hidden = hidden.squeeze()

        # REPARAMETERIZATION
        mean = self.hidden2mean(hidden)
        logv = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logv)
        z = torch.randn([batch_size, self.N_LATENT])
        if torch.cuda.is_available():
            z = z.cuda()
        z = z * std + mean

        # DECODER
        hidden = self.latent2hidden(z)

        if self.bidirectional or self.N_LAYERS > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.N_HIDDEN)
        else:
            hidden = hidden.unsqueeze(0)

        # decoder input
       
        packed_input = rnn_utils.pack_padded_sequence(input_sequence, sorted_lengths.data.tolist(), batch_first=True)

        # decoder forward pass
        outputs, _ = self.decoder_rnn(packed_input, hidden)

        # process outputs
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _,reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b,s,_ = padded_outputs.size()

        # project outputs to vocab
        # logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.size(2))), dim=-1)
        # logp = logp.view(b, s, self.embedding.num_embeddings)
        recons = torch.sigmoid(self.outputs2pred(padded_outputs))
        
        return recons, mean, logv, z

    def inference(self, n=4, z=None):

        if z is None:
            batch_size = n
            if torch.cuda.is_available():
                z = torch.randn([batch_size, self.N_LATENT]).cuda()
        else:
            batch_size = z.size(0)

        hidden = self.latent2hidden(z)

        if self.bidirectional or self.N_LAYERS > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.N_HIDDEN)

        hidden = hidden.unsqueeze(0)

        # required for dynamic stopping of sentence generation
        sequence_idx = torch.arange(0, batch_size, out=self.tensor()).float()  # all idx of batch
        # all idx of batch which are still generating
        sequence_running = torch.arange(0, batch_size, out=self.tensor()).float()
        sequence_mask = torch.ones(batch_size, out=self.tensor()).bool()
        # idx of still generating sequences with respect to current loop
        running_seqs = torch.arange(0, batch_size, out=self.tensor()).float()

        generations = self.tensor(batch_size, self.max_sequence_length,self.N_INPUT).fill_(0).float()
        # generations[:, 1: ,-1] = 1 # Initialize N chords on every time step for the entire batch
        # generations[:, 0, -3] = 1 # Initialize START on every beginnings of sentence
        input_sequence = self.tensor(batch_size, self.max_sequence_length, self.N_INPUT).fill_(0).float().cuda()
        input_sequence[:, 0, -3] = 1 # Initialize START on every beginnings of the sentences
        input_sequence[:, 1:, -1] = 1 # Initialize N on the rest of the sentences
        
        t = 0
        while t < self.max_sequence_length and len(running_seqs) > 0:

            #if t == 0:
            #    if torch.cuda.is_available():
            #        input_sequence = self.tensor(batch_size, 1, self.N_INPUT).fill_(0).float()
            #        input_sequence[:, :, -3] = 1 # Initialize START on every beginnings of sentence
                    #input_sequence = torch.Tensor(batch_size).fill_(self.start_tensor).long().cuda()

            #input_sequence = input_sequence.unsqueeze(1)
            

            output, hidden = self.decoder_rnn(input_sequence[:, :t+1], hidden)

            # create new input_sequence from output

            proba_output = torch.sigmoid(self.outputs2pred(output))
            for i in range(n):
                input_sequence[i,t] = sentence_to_tensor(tensor_to_sentence(proba_output[i], t+1))[t].float()
            
            # save next input
            generations = self._save_sample(generations, input_sequence, sequence_running, t)
            
            # update gloabl running sequence
            sequence_mask[sequence_running.long()] = (input_sequence[:, t, -2] != 1)
            sequence_running = sequence_idx.masked_select(sequence_mask)
            # update local running sequences
            running_mask = (input_sequence[:, t, -2] != 1)
            running_seqs = running_seqs.masked_select(running_mask)
            # prune input and hidden state according to local update
            if len(running_seqs) > 0:
                input_sequence = input_sequence[running_seqs.long()]
                hidden = hidden[:, running_seqs.long()]
                running_seqs = torch.arange(0, len(running_seqs), out=self.tensor()).float()
                

            t += 1

        return generations, z

    def _save_sample(self, save_to, sample, running_seqs, t):
        # select only still running
        running_latest = save_to[running_seqs.long()]
        # update token at position t
        running_latest[:,:t+1] = sample[:,:t+1].data
        # save back
        save_to[running_seqs.long()] = running_latest

        return save_to


In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print('using',dev)
device = torch.device(dev)
model = RVAE(1022).to(device)
model.load_state_dict(torch.load("./E9.pt"))
generations, z = model.inference(5)

using cuda:0
pred one_hot  ['START']
pred one_hot  ['START']
pred one_hot  ['START']
pred one_hot  ['START']
pred one_hot  ['START']
t = 0
shape running = torch.Size([5, 1022, 42])
sample shape = torch.Size([5, 1022, 42])
save_sample ok
update global ok
update local ok
update hidden ok
AAAAAAAAAAAAAAAAAAAAAAAAAAAAH
pred one_hot  ['N', 'N']
et alors ? ton RVAE marche pas Johnny ?
et alors ? ton RVAE marche pas Johnny ?
pred one_hot  ['N', 'N']
et alors ? ton RVAE marche pas Johnny ?
et alors ? ton RVAE marche pas Johnny ?
pred one_hot  ['N', 'N']
et alors ? ton RVAE marche pas Johnny ?
et alors ? ton RVAE marche pas Johnny ?
pred one_hot  ['N', 'N']
et alors ? ton RVAE marche pas Johnny ?
et alors ? ton RVAE marche pas Johnny ?
pred one_hot  ['N', 'N']
et alors ? ton RVAE marche pas Johnny ?
et alors ? ton RVAE marche pas Johnny ?
t = 1
shape running = torch.Size([5, 1022, 42])
sample shape = torch.Size([5, 1022, 42])
save_sample ok
update global ok
update local ok
update hidden ok
AAAA

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Florian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-0cee73c71c9b>", line 9, in <module>
    generations, z = model.inference(5)
  File "<ipython-input-1-48e2dd0c1e2a>", line 170, in inference
    input_sequence[i,t] = sentence_to_tensor(tensor_to_sentence(proba_output[i], t+1))[t].float()
  File "C:\Users\Florian\Documents\jazz-arpeggiator\data_loader_RVAE.py", line 236, in tensor_to_sentence
    elif index_main == TOTAL_MAIN + 1:
  File "C:\Users\Florian\anaconda3\lib\site-packages\torch\tensor.py", line 28, in wrapped
    return f(*args, **kwargs)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Florian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_tracebac

TypeError: object of type 'NoneType' has no len()

In [ ]:
test = torch.Tensor(5,1,4)
len(test.shape)